In [28]:
import pandas as pd
import numpy as np

import datetime
from meteostat import Point, Hourly, Daily

from pandas_profiling import ProfileReport

In [2]:
df = pd.read_csv('cneos_fireball_data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Peak Brightness Date/Time (UT)       929 non-null    object 
 1   Latitude (deg.)                      739 non-null    object 
 2   Longitude (deg.)                     739 non-null    object 
 3   Altitude (km)                        471 non-null    float64
 4   Velocity (km/s)                      276 non-null    float64
 5   vx                                   276 non-null    float64
 6   vy                                   276 non-null    float64
 7   vz                                   276 non-null    float64
 8   Total Radiated Energy (J)            929 non-null    float64
 9   Calculated Total Impact Energy (kt)  929 non-null    float64
dtypes: float64(7), object(3)
memory usage: 72.7+ KB


In [3]:
ProfileReport(df, title='Fireball Report').to_file('load_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# splitting date and time
df['Peak Brightness Date/Time (UT)']= pd.to_datetime(df['Peak Brightness Date/Time (UT)'])

df['date'] = df['Peak Brightness Date/Time (UT)'].dt.date
df['year'] = df['Peak Brightness Date/Time (UT)'].dt.year
df['quarter'] = df['Peak Brightness Date/Time (UT)'].dt.quarter
df['month'] = df['Peak Brightness Date/Time (UT)'].dt.month
df['day'] = df['Peak Brightness Date/Time (UT)'].dt.day
df['time'] = df['Peak Brightness Date/Time (UT)'].dt.time
df['hour'] = df['Peak Brightness Date/Time (UT)'].dt.hour
df['minute'] = df['Peak Brightness Date/Time (UT)'].dt.minute
df['second'] = df['Peak Brightness Date/Time (UT)'].dt.second
df['weekday'] = df['Peak Brightness Date/Time (UT)'].dt.weekday

In [5]:
# changing latitude and longitude to values
def lat(row):
    if row['Latitude (deg.)'] == row['Latitude (deg.)']:
        if row['Latitude (deg.)'][-1] == 'N':
            return float(row['Latitude (deg.)'][:-1])
        elif row['Latitude (deg.)'][-1] == 'S':
            return float('-'+(row['Latitude (deg.)'][:-1]))
    return np.nan

def long(row):
    if row['Latitude (deg.)'] == row['Latitude (deg.)']:
        if row['Longitude (deg.)'][-1] == 'E':
            return float(row['Longitude (deg.)'][:-1])
        elif row['Longitude (deg.)'][-1] == 'W':
            return float('-'+(row['Longitude (deg.)'][:-1]))
    return np.nan

df['Lat'] = df.apply (lambda row: lat(row), axis=1)
df['Long'] = df.apply (lambda row: long(row), axis=1)

In [54]:
#getting climate data
start = datetime.datetime(2021,2,9)
end = datetime.datetime(2021,2,10)
Daily(Point(75.8, -92.8, 31000), start, end).fetch()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun


Peak Brightness Date/Time (UT)         2021-02-09 23:27:29
Latitude (deg.)                                      75.8N
Longitude (deg.)                                     92.8W
Altitude (km)                                         31.0
Velocity (km/s)                                       13.1
vx                                                    -5.2
vy                                                     6.3
vz                                                   -10.3
Total Radiated Energy (J)                    30000000000.0
Calculated Total Impact Energy (kt)                    0.1
date                                            2021-02-09
year                                                  2021
quarter                                                  1
month                                                    2
day                                                      9
time                                              23:27:29
hour                                                    

In [7]:
ProfileReport(df, title='Fireball Report').to_file('fireball_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Note: you may need to restart the kernel to use updated packages.
